In [1]:
from qiskit import Aer
from qiskit.algorithms import VQE
from qiskit.circuit.library import TwoLocal
from qiskit.opflow import X, Y, Z, I, SummedOp

# Define the Hamiltonian
hamiltonian = SummedOp([X ^ X, Y ^ Y, Z ^ Z])

# Choose a variational form
var_form = TwoLocal(hamiltonian.num_qubits, ['ry', 'rz'], 'cz', entanglement='linear')

# Use the Aer simulator backend
backend = Aer.get_backend('statevector_simulator')

# Run VQE
vqe = VQE(hamiltonian, var_form, optimizer=None, quantum_instance=backend)
result = vqe.run()

print("VQE Ground State Energy:", result.optimal_value)
print("VQE Ground State:", result.optimal_point)


C:\Users\luked\AppData\Local\Temp\ipykernel_24408\2659120522.py:2: DeprecationWarning: ``qiskit.algorithms`` has been migrated to an independent package: https://github.com/qiskit-community/qiskit-algorithms. The ``qiskit.algorithms`` import path is deprecated as of qiskit-terra 0.25.0 and will be removed no earlier than 3 months after the release date. Please run ``pip install qiskit_algorithms`` and use ``import qiskit_algorithms`` instead.
  from qiskit.algorithms import VQE
C:\Users\luked\AppData\Local\Temp\ipykernel_24408\2659120522.py:7: DeprecationWarning: The class ``qiskit.opflow.list_ops.summed_op.SummedOp`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/opflow_migration.
  hamiltonian = SummedOp([X ^ X, Y ^ Y, Z ^ Z])
C:\Users\luked\AppData\Local\Temp\ipykernel_24408\2659120522.py:16: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.vqe.VQ

TypeError: __init__() got multiple values for argument 'optimizer'

In [4]:
import numpy as np
from qiskit import QuantumCircuit, execute
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.algorithms.optimizers import SLSQP
from qiskit.circuit.library import PhaseEstimation, TwoLocal
from qiskit.extensions import HamiltonianGate
from qiskit.primitives import Estimator
from qiskit.quantum_info import SparsePauliOp
from qiskit_aer import AerSimulator

H = SparsePauliOp.from_list([('II', -1), ('IZ', 0.3), ('XI', -0.3), ('ZY', -0.01), ('YX', 0.1)])
Hmat = H.to_matrix()

eig = np.linalg.eigvals(Hmat)
print('NUMPY eig',eig)

estimator = Estimator()
optimizer = SLSQP()
ansatz = TwoLocal(rotation_blocks=['ry', 'rz'], entanglement_blocks='cz')

vqe = VQE(estimator, ansatz, optimizer)
result = vqe.compute_minimum_eigenvalue(operator=H)
print('VQE eig', result.eigenvalue)

n_qpe_qbits = 2

U = HamiltonianGate(Hmat, 1, label='H')

# Obtain a solution via QPE
total_qbits = U.num_qubits + n_qpe_qbits
measure_circ = QuantumCircuit(total_qbits, n_qpe_qbits)
measure_circ.h([-1, -2])

qpe = PhaseEstimation(n_qpe_qbits, U)

measure_circ = measure_circ.compose(qpe)
measure_circ.measure(range(n_qpe_qbits), range(n_qpe_qbits))
print(measure_circ.decompose())

backend = AerSimulator(method='statevector')
job = execute(measure_circ, backend, shots=10000)
counts = job.result().get_counts()
print(counts)

max_count = max(counts.items(), key=lambda x: x[1])
print(f'MAX count: {max_count}')

theta = int(max_count[0][::-1], 2) / 2**n_qpe_qbits
print(f'Theta value: {theta}')
print(f'QPE-approximated U-eigenvalue: {np.exp(2*1j*np.pi * theta)}')
print(f'QPE-approximated H-eigenvalue: {-2 * np.pi * theta}')

NUMPY eig [-0.39328755+5.92434709e-18j -0.89      +6.67145251e-17j
 -1.11      -5.23468782e-18j -1.60671245-7.43430782e-17j]
VQE eig -1.6067122688029105
        ┌───┐                                       ┌──────────┐┌─┐   
q_0: ───┤ H ├───────────■───────────────────────────┤1         ├┤M├───
        ├───┤           │                           │  IQFT_dg │└╥┘┌─┐
q_1: ───┤ H ├───────────┼─────────────────■─────────┤0         ├─╫─┤M├
     ┌──┴───┴──┐┌───────┴────────┐┌───────┴────────┐└──────────┘ ║ └╥┘
q_2: ┤ U2(0,π) ├┤0               ├┤0               ├─────────────╫──╫─
     ├─────────┤│  hamiltonian^1 ││  hamiltonian^2 │             ║  ║ 
q_3: ┤ U2(0,π) ├┤1               ├┤1               ├─────────────╫──╫─
     └─────────┘└────────────────┘└────────────────┘             ║  ║ 
c: 2/════════════════════════════════════════════════════════════╩══╩═
                                                                 0  1 
{'10': 8188, '00': 1172, '01': 347, '11': 293}
MAX count: ('10', 8